In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Get to the folder we are at
FOLDERNAME = 'Colab\ Notebooks'
%cd drive/MyDrive/$FOLDERNAME/

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!pip install --target=./ textstat
import textstat
import spacy
import string
import math


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


  Using cached textstat-0.7.3-py3-none-any.whl (105 kB)
  Using cached pyphen-0.14.0-py3-none-any.whl (2.0 MB)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
!ls

# file_path = 'train_esssays_v2.csv'
file_path = 'train_V2_csv'

!echo $file_path

raw_data = pd.read_csv(file_path)

train = pd.read_csv(file_path)

file_path = 'test_V2_csv'
test = pd.read_csv(file_path)

file_path = 'val_V2_csv'
val = pd.read_csv(file_path)


'BERT(1).ipynb'       'LSTM(1)'   pyphen-0.14.0.dist-info   textstat-0.7.3.dist-info   val_V2_csv
 bert_model.pth       'LSTM(2)'   test_essays_v1.csv	    train_esssays_v2.csv
 bert_submission.csv   old_data   test_V2_csv		    train_esssays_v2.gsheet
 Jay		       pyphen	  textstat		    train_V2_csv
train_V2_csv


In [ ]:
print(f'train: {train.shape}')
print(f'test: {test.shape}')
print(f'val: {val.shape}')

train: (28715, 6)
test: (8974, 6)
val: (7179, 6)


In [ ]:
train.head()

,Unnamed: 0,text,label,prompt_name,source,RDizzl3_seven
0,31857,The pursuit of success is often portrayed as ...,1,Seeking multiple opinions,mistral7binstruct_v2,False
1,39559,"Sure, here's my essay on the benefits of offer...",1,Distance learning,llama2_chat,False
2,35407,The Development of Driverless Cars\n\nWhile dr...,1,Driverless cars,darragh_claude_v7,True
3,24411,Today technology is arriving into the schools....,0,Distance learning,persuade_corpus,False
4,26433,The advantages of limiting car usage are becom...,1,Car-free cities,radek_500,True


In [ ]:
train.prompt_name.value_counts()

Distance learning                        3545
Seeking multiple opinions                3294
Car-free cities                          3070
Does the electoral college work?         2838
Mandatory extracurricular activities     1992
Facial action coding system              1952
Summer projects                          1735
Exploring Venus                          1418
Driverless cars                          1414
Grades for extracurricular activities    1346
Community service                        1329
Cell phones at school                    1326
The Face on Mars                         1241
"A Cowboy Who Rode the Waves"            1177
Phones and driving                       1038
Name: prompt_name, dtype: int64

In [ ]:
val.head()

,Unnamed: 0,text,label,prompt_name,source,RDizzl3_seven
0,31643,"Dear Principal Johnson, \n\nI am writing to ex...",1,Grades for extracurricular activities,darragh_claude_v7,False
1,13906,"Dear TEACHER_NAME,\n\nI have heard you are mak...",0,Community service,persuade_corpus,False
2,3174,Many schools require students to do projects i...,0,Summer projects,persuade_corpus,False
3,7235,I agree that you should have to do extracurric...,0,Mandatory extracurricular activities,persuade_corpus,False
4,15522,Have you ever wondered how many kids would not...,0,Grades for extracurricular activities,persuade_corpus,False


In [ ]:
# feature: prompt在text中出現頻率(如何計算)、num_unique_words、num_words、num_sentences、num_slash("\n")、paragraph_count("\n\n")、syntax_count、錯字比例、詞性???、readability_scores?、_flesch_reading_ease???


In [ ]:
class Preprocessor:
    def __init__(self,
                ) -> None:
        self.STOP_WORDS = set(nltk.corpus.stopwords.words('english'))
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.nltk_tokenizer = nltk.word_tokenize
        self.nltk_pos_tag = nltk.pos_tag  # 辭性標籤
        self.ner_keys = set()


    def count_text_length(self, df: pd.DataFrame, col: str) -> pd.Series:
        """ text length """
        return df[col].progress_apply(lambda x: len(self.nltk_tokenizer(x)))



    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]


    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)

        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))

        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    def spelling(self, text):

        wordlist=text.split()
        amount_miss = len(list(self.speller.unknown(wordlist)))

        return amount_miss
    def add_textstat_features(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
        df[f"{col}_flesch_reading_ease"] = df[col].apply(textstat.flesch_reading_ease) # Returns the Flesch Reading Ease Score
        df[f"{col}_flesch_kincaid_grade"] = df[col].apply(textstat.flesch_kincaid_grade) # defficulty
        df[f"{col}_smog_index"] = df[col].apply(textstat.smog_index) # defficulty
        df[f"{col}_coleman_liau_index"] = df[col].apply(textstat.coleman_liau_index) # defficulty
        df[f"{col}_automated_readability_index"] = df[col].apply(textstat.automated_readability_index) # defficulty
        df[f"{col}_dale_chall_readability_score"] = df[col].apply(textstat.dale_chall_readability_score) # defficulty
        df[f"{col}_difficult_words"] = df[col].apply(textstat.difficult_words)
        df[f"{col}_linsear_write_formula"] = df[col].apply(textstat.linsear_write_formula) # defficulty
        df[f"{col}_gunning_fog"] = df[col].apply(textstat.gunning_fog) # defficulty
        df[f"{col}_text_standard"] = df[col].apply(lambda x: textstat.text_standard(x, float_output=True)) # defficulty
        return df

    def get_readability_scores(self, text: str) -> dict:
        r = Readability(text)

        scores = {
            'flesch_kincaid': None,
            'flesch': None,
            'gunning_fog': None,
            'coleman_liau': None,
            'dale_chall': None,
            'ari': None,
            'linsear_write': None,
            'smog': None,
            'spache': None
        }

        try: scores['flesch_kincaid'] = r.flesch_kincaid().score
        except: scores['flesch_kincaid'] = np.NaN

        try: scores['flesch'] = r.flesch().score
        except: scores['flesch'] = np.NaN

        try: scores['gunning_fog'] = r.gunning_fog().score
        except: scores['gunning_fog'] = np.NaN

        try: scores['coleman_liau'] = r.coleman_liau().score
        except: scores['coleman_liau'] = np.NaN

        try: scores['dale_chall'] = r.dale_chall().score
        except: scores['dale_chall'] = np.NaN

        try: scores['ari'] = r.ari().score
        except: scores['ari'] = np.NaN

        try: scores['linsear_write'] = r.linsear_write().score
        except: scores['linsear_write'] = np.NaN

        try: scores['smog'] = r.smog().score
        except: scores['smog'] = np.NaN

        try: scores['spache'] = r.spache().score
        except: scores['spache'] = np.NaN

        return scores

    def get_stat_features(self,df, text_col="text"):
        df["num_unique_words"] = df[text_col].apply(lambda x: len(set(x.split())))
        df["num_words"] = df[text_col].apply(lambda x: len(x.split()))
        df["num_sentences"] = df[text_col].apply(lambda x: len(x.split('.')))
        df["isupper"] = df[text_col].apply(lambda x: x[0].isupper())
        df["mean_num_words"] = df[text_col].apply(lambda x: np.mean([len(e.split()) for e in x.split('.')]))
        df["mean_num_unique_words"] = df[text_col].apply(lambda x: np.mean([len(set(e.split())) for e in x.split('.')]))
        df["num_slash"] = df[text_col].apply(lambda x: x.count("\n"))
        df["paragraph_count"] = df[text_col].apply(lambda x: x.count("\n\n"))
        df["upper_count"] = df[text_col].apply(lambda x: np.sum([w.isupper() for w in x.split()])/len(x.split()))
        df["syntax_count"] = df[text_col].apply(lambda x: x.count(",") + x.count("-") + x.count(";") + x.count(":"))
        df["is_end_with_dot"] = df[text_col].apply(lambda x: int(x[-1] == "."))
        return df
    def pos_count(self,sent):
        nn_count = 0   #Noun
        pr_count = 0   #Pronoun
        vb_count = 0   #Verb
        jj_count = 0   #Adjective
        uh_count = 0   #Interjection
        cd_count = 0   #Numerics

        sent = nltk.word_tokenize(sent)
        sent = nltk.pos_tag(sent)

        for token in sent:
            if token[1] in ['NN','NNP','NNS']:
                nn_count += 1

            if token[1] in ['PRP','PRP$']:
                pr_count += 1

            if token[1] in ['VB','VBD','VBG','VBN','VBP','VBZ']:
                vb_count += 1

            if token[1] in ['JJ','JJR','JJS']:
                jj_count += 1

            if token[1] in ['UH']:
                uh_count += 1

            if token[1] in ['CD']:
                cd_count += 1

        return pd.Series([nn_count, pr_count, vb_count, jj_count, uh_count, cd_count],
                     index=['nn_count', 'pr_count', 'vb_count', 'jj_count', 'uh_count', 'cd_count'])



    def run(self,
            df: pd.DataFrame,

        ) -> pd.DataFrame:

        df["text_length"] = df["text"].apply(lambda x: len(self.nltk_tokenizer(x)))
        df["text_tokens"] = df["text"].apply(self.nltk_tokenizer)
        #df["entropy_value"] = df["text"].apply(self.get_entropy)

        # merge prompts and summaries
        df = self.get_stat_features(df)
        pos_df = df['text'].apply(self.pos_count)
        df = pd.concat([df, pos_df], axis=1)

        # after merge preprocess
        df = self.add_textstat_features(df, 'text')

        return df

preprocessor = Preprocessor()
train_df = preprocessor.run(train)
test_df = preprocessor.run(test)
val_df = preprocessor.run(val)

In [ ]:


# def get_entropy(text_token):
#   entropy_value = stats.entropy(text_token)
#   return entropy_value


In [ ]:
train_label = train.pop('label')
test_label = test.pop('label')
val_label = val.pop('label')

In [ ]:
train_df.pop('label')
test_df.pop('label')
val_df.pop('label')

0       1
1       0
2       0
3       0
4       0
       ..
7174    0
7175    0
7176    0
7177    0
7178    0
Name: label, Length: 7179, dtype: int64

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28715 entries, 0 to 28714
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         28715 non-null  int64  
 1   text                               28715 non-null  object 
 2   prompt_name                        28715 non-null  object 
 3   source                             28715 non-null  object 
 4   RDizzl3_seven                      28715 non-null  bool   
 5   text_length                        28715 non-null  int64  
 6   text_tokens                        28715 non-null  object 
 7   num_unique_words                   28715 non-null  int64  
 8   num_words                          28715 non-null  int64  
 9   num_sentences                      28715 non-null  int64  
 10  isupper                            28715 non-null  bool   
 11  mean_num_words                     28715 non-null  flo

In [ ]:
train_df.head(3)

,Unnamed: 0,text,prompt_name,source,RDizzl3_seven,text_length,text_tokens,num_unique_words,num_words,num_sentences,...,text_flesch_reading_ease,text_flesch_kincaid_grade,text_smog_index,text_coleman_liau_index,text_automated_readability_index,text_dale_chall_readability_score,text_difficult_words,text_linsear_write_formula,text_gunning_fog,text_text_standard
0,31857,The pursuit of success is often portrayed as ...,Seeking multiple opinions,mistral7binstruct_v2,False,450,"[The, pursuit, of, success, is, often, portray...",196,395,18,...,40.79,13.0,16.1,12.82,14.6,8.48,80,11.833333,14.23,13.0
1,39559,"Sure, here's my essay on the benefits of offer...",Distance learning,llama2_chat,False,437,"[Sure, ,, here, 's, my, essay, on, the, benefi...",171,389,19,...,59.13,10.1,11.5,11.49,12.8,7.41,56,15.750000,11.08,12.0
2,35407,The Development of Driverless Cars\n\nWhile dr...,Driverless cars,darragh_claude_v7,True,288,"[The, Development, of, Driverless, Cars, While...",192,261,12,...,25.69,14.7,15.9,17.64,17.5,10.68,92,13.800000,16.32,18.0


In [ ]:
train_df.pop('Unnamed: 0')
train_df.pop('text')
train_df.pop('prompt_name')
train_df.pop('source')
train_df.pop('RDizzl3_seven')
train_df.pop('text_tokens')
train_df.pop('num_unique_words')
train_df.pop('num_words')

test_df.pop('Unnamed: 0')
test_df.pop('text')
test_df.pop('prompt_name')
test_df.pop('source')
test_df.pop('RDizzl3_seven')
test_df.pop('text_tokens')
test_df.pop('num_unique_words')
test_df.pop('num_words')

val_df.pop('Unnamed: 0')
val_df.pop('text')
val_df.pop('prompt_name')
val_df.pop('source')
val_df.pop('RDizzl3_seven')
val_df.pop('text_tokens')
val_df.pop('num_unique_words')
val_df.pop('num_words')

0       328
1       312
2       641
3       251
4       373
       ... 
7174    453
7175    165
7176    273
7177    283
7178    490
Name: num_words, Length: 7179, dtype: int64

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28715 entries, 0 to 28714
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   text_length                        28715 non-null  int64  
 1   num_sentences                      28715 non-null  int64  
 2   isupper                            28715 non-null  bool   
 3   mean_num_words                     28715 non-null  float64
 4   mean_num_unique_words              28715 non-null  float64
 5   num_slash                          28715 non-null  int64  
 6   paragraph_count                    28715 non-null  int64  
 7   upper_count                        28715 non-null  float64
 8   syntax_count                       28715 non-null  int64  
 9   is_end_with_dot                    28715 non-null  int64  
 10  nn_count                           28715 non-null  int64  
 11  pr_count                           28715 non-null  int

In [ ]:
from sklearn import preprocessing, linear_model, metrics, model_selection, svm, ensemble, metrics

# degree 1
h = linear_model.LogisticRegression(max_iter=2500)
classifier = h.fit(train_df, train_label)
predictions = classifier.predict(train_df)
roc_auc = metrics.roc_auc_score(train_label, predictions)
print('Degree 1 Training Auc:', round(roc_auc, 4))
predictions = classifier.predict(val_df)
roc_auc = metrics.roc_auc_score(val_label, predictions)
print('Degree 1 Val Auc:', round(roc_auc, 4))
predictions = classifier.predict(test_df)
roc_auc = metrics.roc_auc_score(test_label, predictions)
print('Degree 1 Test Auc:', round(roc_auc, 4))

Degree 1 Training Auc: 0.9104
Degree 1 Val Auc: 0.92
Degree 1 Test Auc: 0.9114


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# degree 2 (best)
poly_fea_extractor = preprocessing.PolynomialFeatures(degree=2)
df_train_degree2 = poly_fea_extractor.fit_transform(train_df)
classifier = h.fit(df_train_degree2, train_label)
predictions = classifier.predict(df_train_degree2)
roc_auc = metrics.roc_auc_score(train_label, predictions)
print('Degree 2 Training Auc:', round(roc_auc, 4))
df_val_degree2 = poly_fea_extractor.fit_transform(val_df)
classifier = h.fit(df_val_degree2, val_label)
predictions = classifier.predict(df_val_degree2)
roc_auc = metrics.roc_auc_score(val_label, predictions)
print('Degree 2 Val Auc:', round(roc_auc, 4))
df_test_degree2 = poly_fea_extractor.fit_transform(test_df)
classifier = h.fit(df_test_degree2, test_label)
predictions = classifier.predict(df_test_degree2)
roc_auc = metrics.roc_auc_score(test_label, predictions)
print('Degree 2 Test Auc:', round(roc_auc, 4))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Degree 2 Training Auc: 0.9542


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Degree 2 Val Auc: 0.9568
Degree 2 Test Auc: 0.955


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from xgboost import XGBClassifier

xgboostModel = XGBClassifier()
xgboostModel = xgboostModel.fit(train_df, train_label)
predicted = xgboostModel.predict(train_df)
roc_auc = metrics.roc_auc_score(train_label, predicted)
print('xgboost Training Auc:', round(roc_auc, 4))
predicted = xgboostModel.predict(val_df)
roc_auc = metrics.roc_auc_score(val_label, predicted)
print('xgboost Val Auc:', round(roc_auc, 4))
predicted = xgboostModel.predict(test_df)
roc_auc = metrics.roc_auc_score(test_label, predicted)
print('xgboost Test Auc:', round(roc_auc, 4))


xgboost Training Auc: 0.9975
xgboost Val Auc: 0.9625
xgboost Test Auc: 0.9617
